In [1]:
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from torch_geometric.datasets import Planetoid

dataset = PygNodePropPredDataset(name='ogbn-arxiv')
# dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

In [3]:
from torch_geometric.nn.models import Node2Vec
from torch.optim import SparseAdam

model = Node2Vec(
    data.edge_index, 
    embedding_dim=128, 
    walk_length=32,
    context_size=16, 
    walks_per_node=16, 
    num_negative_samples=1,
    sparse=True    
    ).to("cpu")

loader = model.loader(batch_size=128, shuffle=True, num_workers=8)
optimizer = SparseAdam(list(model.parameters()), lr=0.01)

In [4]:
def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to("cpu"), neg_rw.to("cpu"))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [5]:
for epoch in range(1):
    loss = train()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

Epoch: 00, Loss: 2.2555


In [6]:
evaluator = Evaluator(name='ogbn-arxiv')

In [20]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv')
data = dataset[0]
x = data.x.to("cpu")
y_true = data.y.to("cpu")
split_idx = dataset.get_idx_split()

In [19]:
import torch
@torch.no_grad()
def test(model, x, y_true, split_idx, evaluator):
    model.eval()

    out = model(x)
    print(out)